In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kapaa,US,2022-07-05 14:14:34,22.0752,-159.3190,75.52,78,75,17.27
1,1,Rikitea,PF,2022-07-05 14:14:34,-23.1203,-134.9692,74.57,79,33,11.52
2,2,Lavrentiya,RU,2022-07-05 14:14:34,65.5833,-171.0000,45.03,95,97,6.24
3,3,Punta Arenas,CL,2022-07-05 14:10:54,-53.1500,-70.9167,26.71,93,40,5.75
4,4,Provideniya,RU,2022-07-05 14:14:35,64.3833,-173.3000,46.09,97,100,5.30


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [15]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(0.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp],
                                dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Heatmap of cloudiness
locations = city_data_df[["Lat", "Lng"]]
cloud = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloud, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Heatmap of windspeed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [74]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

ValueError: Can only compare identically-labeled Series objects

In [75]:
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 594
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     159 non-null    int64  
 1   City        159 non-null    object 
 2   Country     159 non-null    object 
 3   Date        159 non-null    object 
 4   Lat         159 non-null    float64
 5   Lng         159 non-null    float64
 6   Max Temp    159 non-null    float64
 7   Humidity    159 non-null    int64  
 8   Cloudiness  159 non-null    int64  
 9   Wind Speed  159 non-null    float64
dtypes: float64(4), int64(3), object(3)
memory usage: 17.7+ KB


In [76]:
preferred_cities_df.count()

City_ID       159
City          159
Country       159
Date          159
Lat           159
Lng           159
Max Temp      159
Humidity      159
Cloudiness    159
Wind Speed    159
dtype: int64

In [77]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kapaa,US,75.52,22.0752,-159.3190,
1,Rikitea,PF,74.57,-23.1203,-134.9692,
10,Ryotsu,JP,74.91,38.0833,138.4333,
12,Avarua,CK,73.45,-21.2078,-159.7750,
16,Mahebourg,MU,71.89,-20.4081,57.7000,
23,Hilo,US,79.29,19.7297,-155.0900,
24,Katsuura,JP,74.89,35.1333,140.3000,
26,Auki,SB,77.11,-8.7676,160.7034,
35,Souillac,MU,71.98,-20.5167,57.5167,
44,Catuday,PH,78.33,16.2923,119.8062,


In [78]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [79]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("hotel not found...skipping")

hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping
hotel not found...skipping


In [80]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kapaa,US,75.52,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
1,Rikitea,PF,74.57,-23.1203,-134.9692,People ThankYou
10,Ryotsu,JP,74.91,38.0833,138.4333,Sadokamoko Onsen Kohanno Yado Yoshidaya
12,Avarua,CK,73.45,-21.2078,-159.7750,Paradise Inn
16,Mahebourg,MU,71.89,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [81]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 594
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        159 non-null    object 
 1   Country     159 non-null    object 
 2   Max Temp    159 non-null    float64
 3   Lat         159 non-null    float64
 4   Lng         159 non-null    float64
 5   Hotel Name  159 non-null    object 
dtypes: float64(3), object(3)
memory usage: 12.8+ KB


In [82]:
hotel_df.count()

City          159
Country       159
Max Temp      159
Lat           159
Lng           159
Hotel Name    159
dtype: int64

In [83]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [86]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [89]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [90]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [91]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))